In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import sqlite3

conn = sqlite3.connect('../train_mTSP.sqlite3')
instances = pd.read_sql_query("SELECT * FROM instances", conn)
algorithms = pd.read_sql_query("SELECT * FROM algorithms", conn)
conn.close()

data = pd.merge(instances, algorithms, on="instance_id")

# Filter out branch and cut for large instances (they tend to give bad distributions)
data = data[~((data['strategy'] == 'branch and cut') & (data['nr_cities'] > 50))]

# Compute composite score 
data['composite_score'] = (
    data['normalized_cost'] * 0.45 + 
    data['time_taken'] * 0.35 + 
    data['distance_gap'] * 0.2
)

# Select best strategy per instance
best_strategies = data.loc[data.groupby('instance_id')['composite_score'].idxmin()]

# Select features and labels
features = best_strategies[['nr_cities', 'nr_salesmen', 'average_distance', 'stddev_distance', 'density',
                            'salesmen_ratio', 'bounding_box_area', 'aspect_ratio', 'spread',
                            'cluster_compactness', 'mst_total_length', 'entropy_distance_matrix']]
labels = pd.get_dummies(best_strategies['strategy'])

# Normalize features
scaler = StandardScaler()
features = scaler.fit_transform(features)

x_train = torch.tensor(features, dtype=torch.float32)
y_train = torch.tensor(labels.values, dtype=torch.float32)


conn = sqlite3.connect('../validation_mTSP.sqlite3')
validation_instances = pd.read_sql_query("SELECT * FROM instances", conn)
validation_algorithms = pd.read_sql_query("SELECT * FROM algorithms", conn)
conn.close()

validation_data = pd.merge(validation_instances, validation_algorithms, on="instance_id")
validation_data = validation_data[~((validation_data['strategy'] == 'branch and cut') & (validation_data['nr_cities'] > 50))]

# Compute composite score for validation data
validation_data['composite_score'] = (
    validation_data['normalized_cost'] * 0.45 + 
    validation_data['time_taken'] * 0.35 + 
    validation_data['distance_gap'] * 0.2
)

# Select best strategy per validation instance
validation_best = validation_data.loc[validation_data.groupby('instance_id')['composite_score'].idxmin()]

# Select features and labels for validation
validation_features = validation_best[['nr_cities', 'nr_salesmen', 'average_distance', 'stddev_distance', 'density',
                                       'salesmen_ratio', 'bounding_box_area', 'aspect_ratio', 'spread',
                                       'cluster_compactness', 'mst_total_length', 'entropy_distance_matrix']]
validation_labels = pd.get_dummies(validation_best['strategy'])

# Normalize validation features
validation_features = scaler.transform(validation_features)

# Ensure validation labels have the same columns as training labels
for col in labels.columns:
    if col not in validation_labels.columns:
        validation_labels[col] = 0
validation_labels = validation_labels[labels.columns]  # ensure same order
validation_labels = validation_labels.astype(float)

x_val = torch.tensor(validation_features, dtype=torch.float32)
y_val = torch.tensor(validation_labels.values, dtype=torch.float32)

# Neural Network Model 
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(32, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return self.softmax(x)

input_size = x_train.shape[1]
output_size = y_train.shape[1]
model = NeuralNetwork(input_size, output_size)

# Calculate class weights for imbalanced classes so that the model pays more attention to underrepresented classes
class_counts = best_strategies['strategy'].value_counts()
class_weights = 1.0 / class_counts
class_weights_dict = {strategy: class_weights[strategy] for strategy in class_counts.index}
weights = torch.tensor([class_weights_dict[label] for label in labels.columns], dtype=torch.float32)
criterion = nn.CrossEntropyLoss(weight=weights)
# criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

best_val_loss = float('inf')
patience = 25  # number of epochs to wait for improvement
trigger_times = 0

# Train
epochs = 300
batch_size = 32
for epoch in range(epochs):
    model.train()
    permutation = torch.randperm(x_train.size(0))
    for i in range(0, x_train.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_X, batch_y = x_train[indices], y_train[indices]

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, torch.argmax(batch_y, dim=1))
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(x_val)
        val_loss = criterion(val_outputs, torch.argmax(y_val, dim=1))
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

    # Stop if no improvement in validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trigger_times = 0
        torch.save(model.state_dict(), 'best_model.pth')  
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print("Early stopping triggered!")
            break

    scheduler.step()

# torch.save(model.state_dict(), 'algorithm_selector_model.pth')


Epoch 1/300, Train Loss: 1.3915, Val Loss: 1.3404
Epoch 2/300, Train Loss: 1.3824, Val Loss: 1.2840
Epoch 3/300, Train Loss: 1.3566, Val Loss: 1.2379
Epoch 4/300, Train Loss: 1.4024, Val Loss: 1.2033
Epoch 5/300, Train Loss: 1.3419, Val Loss: 1.1789
Epoch 6/300, Train Loss: 1.3616, Val Loss: 1.1601
Epoch 7/300, Train Loss: 1.0826, Val Loss: 1.1388
Epoch 8/300, Train Loss: 1.2867, Val Loss: 1.1218
Epoch 9/300, Train Loss: 1.2456, Val Loss: 1.1063
Epoch 10/300, Train Loss: 1.4091, Val Loss: 1.0874
Epoch 11/300, Train Loss: 1.2494, Val Loss: 1.0915
Epoch 12/300, Train Loss: 1.3126, Val Loss: 1.0960
Epoch 13/300, Train Loss: 1.1816, Val Loss: 1.0759
Epoch 14/300, Train Loss: 1.2814, Val Loss: 1.0809
Epoch 15/300, Train Loss: 1.5127, Val Loss: 1.0918
Epoch 16/300, Train Loss: 1.1970, Val Loss: 1.0845
Epoch 17/300, Train Loss: 1.1311, Val Loss: 1.0770
Epoch 18/300, Train Loss: 1.0164, Val Loss: 1.0789
Epoch 19/300, Train Loss: 0.9905, Val Loss: 1.0756
Epoch 20/300, Train Loss: 1.1370, Val Lo

In [2]:
print(best_strategies['strategy'].value_counts())

with torch.no_grad():
    train_outputs = model(x_train)
    train_predictions = torch.argmax(train_outputs, dim=1)
    print(pd.Series(train_predictions.numpy()).value_counts())

strategy
Ant Colony        558
KMeans-Greedy     449
Greedy             64
Branch and Cut     18
Name: count, dtype: int64
0    523
3    268
2    256
1     42
Name: count, dtype: int64


In [3]:
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import pandas as pd
import sqlite3

input_size = 12  # nr of features
output_size = 4  # nr of algorithms
model = NeuralNetwork(input_size, output_size)
model.load_state_dict(torch.load('best_model.pth'))
model.eval()


conn = sqlite3.connect('../test_mTSP.sqlite3')
test_instances = pd.read_sql_query("SELECT * FROM instances", conn)
conn.close()


test_features = test_instances[['nr_cities', 'nr_salesmen', 'average_distance', 'stddev_distance', 'density',
                                 'salesmen_ratio', 'bounding_box_area', 'aspect_ratio', 'spread',
                                 'cluster_compactness', 'mst_total_length', 'entropy_distance_matrix']]

scaler = StandardScaler()
test_features = scaler.fit_transform(test_features)  # ensure the scaler is consistent with training

x_test = torch.tensor(test_features, dtype=torch.float32)

# Predict the best algorithm for each instance
with torch.no_grad():
    predictions = model(x_test)
    predicted_classes = torch.argmax(predictions, dim=1)

# Map predicted classes to algorithm names
algorithm_mapping = {0: 'branch and cut', 1: 'greedy', 2: 'kmeans greedy', 3: 'ant colony optimization'}
predicted_algorithms = [algorithm_mapping[class_idx.item()] for class_idx in predicted_classes]


test_instances['predicted_algorithm'] = predicted_algorithms
print(test_instances[['instance_id', 'predicted_algorithm']])
test_instances[['instance_id', 'predicted_algorithm']].to_csv('predicted_algorithms.csv', index=False)

     instance_id      predicted_algorithm
0              1           branch and cut
1              2           branch and cut
2              3           branch and cut
3              4           branch and cut
4              5           branch and cut
..           ...                      ...
96            97  ant colony optimization
97            98  ant colony optimization
98            99  ant colony optimization
99           100  ant colony optimization
100          101  ant colony optimization

[101 rows x 2 columns]
